In [ ]:
from bs4 import BeautifulSoup
import requests
import csv
import time
import pandas as pd

data = []

url = 'https://www.capterra.co.uk/reviews/162332/cluvio'

url = url+'?page=1'

source = requests.get(url).text
time.sleep(2)
soup = BeautifulSoup(source,'lxml')

try:
    last_page_button = int(soup.find_all('a', class_='page-link')[-2].text)
except:
    last_page_button = 1

for i in range(1,last_page_button + 1):
    
    url = url.rstrip(url[-1])
    url = '{}{}'.format(url,i)
    source = requests.get(url).text
    time.sleep(2)
    soup = BeautifulSoup(source,'lxml')
    
    reviewcard = soup.find('div',class_='col-lg-6 pr-lg-5')
    
    for reviewcard in soup.find_all('div',class_="i18n-translation_container review-card pt-4 py-3 py-md-5 border-bottom border-primary"):
    
        ### personal details ### 
    
        try:  
            name = reviewcard.h5.text.split('\n')
            name = ''.join(name)
    
        except Exception as e:
            name = None
    
        print(name)
    
        try:
            jobtitle = reviewcard.find('div',class_='text-muted small').text
        except:
            jobtitle = None
    
        print(jobtitle)
    
        try:
            companytype = reviewcard.find('div',class_='text-italic text-muted small').text.split('\n')
            companytype = ''.join(companytype)
        except:
            companytype = None
    
        print(companytype)
    
        try:
            uselength = reviewcard.find('div',class_='font-weight-light text-muted small').text.split('\n')
            uselength = ''.join(uselength).split(': ')[1]
        except:
            uselength = None
    
        print(uselength)
    
    
        ### ratings ###
    
        ratings = reviewcard.find('div',class_='pb-2 py-md-4 mb-4')
    
        try:
            overall_rating = ratings.find('span',class_="text-nowrap d-inline-block text-right review-stars__text font-weight-bold").text.split('\n')[1]
        except:
            overall_rating = 'Unrated'
    
        print(overall_rating)
    
    
        row2 = ratings.find('li',class_="d-flex py-2 row")
    
        try:
            ease_of_use = row2.find('span',class_="text-nowrap d-inline-block text-right review-stars__text font-weight-bold").text.split('\n')[1]
        except:
            ease_of_use = 'Unrated'
    
        print(ease_of_use)
    
        row3 = ratings.find_all("li",{"class":"d-flex py-2 row"})[1]
    
        try:
            features = row3.find('span',class_="text-nowrap d-inline-block text-right review-stars__text font-weight-bold").text.split('\n')[1]
        except:
            features = 'Unrated'
        print(features)
    
        row4 = ratings.find_all("li",{"class":"d-flex py-2 row"})[2]
    
        try:
            customer_support = row4.find('span',class_="text-nowrap d-inline-block text-right review-stars__text font-weight-bold").text.split('\n')[1]
        except:
            customer_support = 'Unrated'
    
        print(customer_support)
    
        row5 = ratings.find_all("li",{"class":"d-flex py-2 row"})[3]
    
        try:
            value = row5.find('span',class_="text-nowrap d-inline-block text-right review-stars__text font-weight-bold").text.split('\n')[1]
        except: 
            value = 'Unrated'
    
        print(value)
    
        row6 = ratings.find_all("li",{"class":"d-flex py-2 row"})[4]
    
        try:
            recommend = row6.find('div',class_="text-nowrap d-inline-block text-right review-bar__text").text.split('/')[0]
        except:
            recommend = 'Unrated'
    
        print(recommend)
    
        ### Review Text ###
    
        reviews = reviewcard.find('div',class_='col-lg-6 pl-lg-5')
    
        try:
            headline = reviews.h3.text.split('"')[1]
        except:
            headline = None
        print(headline)
    
    
    
        textlist = reviews.p.text.split('\n')
        textlist = ''.join(textlist).split(':')
    
        if textlist[0] == 'Comments':
            try:
                comments = reviews.p.text.split('Comments:')[1].split('\n')
                comments = ' '.join(comments).rstrip().lstrip()
            except:
                comments = None
    
            try:
                pros = reviews.find_all('p')[1].text.split('Pros:')[1].split('\n')
                pros = ' '.join(pros).rstrip().lstrip()
            except:
                pros = None
    
            try:
                cons = reviews.find_all('p')[2].text.split('Cons:')[1].split('\n')
                cons = ' '.join(cons).rstrip().lstrip()
            except:
                cons = None  
    
        else:
    
            comments = None
    
            try:
                pros = reviews.find_all('p')[0].text.split('Pros:')[1].split('\n')
                pros = ' '.join(pros).rstrip().lstrip()
            except:
                pros = None
    
            try:
                cons = reviews.find_all('p')[1].text.split('Cons:')[1].split('\n')
                cons = ' '.join(cons).rstrip().lstrip()
            except:
                cons = None 
    
        print(comments)
        print(pros)
        print(cons)
    
        try:
            date = reviews.find('div',class_='d-lg-none').text.split()[-1]
        except:
            date = None
        print(date)
        
        print()
        
        row = [name,jobtitle,companytype,uselength,
                             overall_rating,ease_of_use,features,
                             customer_support,value,recommend,headline,
                             comments,pros,cons,date]
        data.append(row)
        
df = pd.DataFrame(data, columns = ['name','jobtitle','companytype','uselength',
                     'overall_rating','ease_of_use','features',
                     'customer_support','value','recommend','headline',
                    'comments','pros','cons','date'])

df.to_csv('capterra_scrape.csv',index=False)